In [1]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import *
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import IBMQ, Aer, execute
# Loading your IBM Q account(s)

ModuleNotFoundError: No module named 'qiskit'

## Example: Find the data from qRAM
Prepare a qRAM of $n$-addresses in which the numbers $k_0, k_1, .. , k_{n-1}$ are stored in this order.  
Find the address in which the number $m$ is stored using Grover's algorithm.  
- $n = 4$
- $k = [1,2,5,7]$
- $m = 5$

### qRAM operation.
Here we show a circuit example of qRAM.

In [2]:
address = QuantumRegister(2)
data = QuantumRegister(3)
oracle = QuantumRegister(1)
c = ClassicalRegister(2)
qc = QuantumCircuit(address,data,oracle,c)

# address preparation
qc.h([address[0],address[1]])
qc.x(oracle[0])
qc.h(oracle[0])
qc.barrier()

##Qram part

# address 0 -> data = 1 (001)
qc.x([address[0],address[1]])
qc.ccx(address[0],address[1],data[2]) 
qc.x([address[0],address[1]])
qc.barrier()

# address 1 -> data = 2 (010)
qc.x(address[0])
qc.ccx(address[0],address[1],data[1])
qc.x(address[0])
qc.barrier()

# address 2 -> data = 5 (101)
qc.x(address[1])
qc.ccx(address[0],address[1],data[2])
qc.ccx(address[0],address[1],data[0])
qc.x(address[1])
qc.barrier()
# address 3 -> data = 7 (111)
qc.ccx(address[0],address[1],data[2])
qc.ccx(address[0],address[1],data[1])
qc.ccx(address[0],address[1],data[0])
qc.barrier()

#flip phase of m = 5 (101)
qc.x(data[1])
qc.mct([data[0], data[1], data[2]], oracle[0], mode='noancilla')
qc.barrier()

## Reverse Qram part

# address 3 -> data = 7 (111)
qc.ccx(address[0],address[1],data[2])
qc.ccx(address[0],address[1],data[1])
qc.ccx(address[0],address[1],data[0])
qc.barrier()

# address 2 -> data = 5 (101)
qc.x(address[1])
qc.ccx(address[0],address[1],data[2])
qc.ccx(address[0],address[1],data[0])
qc.x(address[1])
qc.barrier()

# address 1 -> data = 2 (010)
qc.x(address[0])
qc.ccx(address[0],address[1],data[1])
qc.x(address[0])
qc.barrier()

# address 0 -> data = 1 (001)
qc.x([address[0],address[1]])
qc.ccx(address[0],address[1],data[2])
qc.x([address[0],address[1]])
qc.barrier()


#diffusion
qc.h(address[:2])
qc.x(address[:2])
qc.h(address[1])
qc.cx(address[0], address[1])
qc.h(address[1])
qc.x(address[:2])
qc.h(address[:2])

#Measure the address
qc.measure(address[0:2], c[0:2])

# Reverse the output string.
qc = qc.reverse_bits()

#backend = provider.get_backend('ibmq_qasm_simulator')
backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend=backend, shots=80000, seed_simulator=12345, backend_options={"fusion_enable":True})
result = job.result()
count =result.get_counts()
print(count)

qc.draw(output='mpl')

NameError: name 'QuantumRegister' is not defined